In [2]:
import pandas as pd
import numpy as np

In [25]:
# load data
#full_data = pd.read_csv("../data/questions.csv")

# split dataset
def split_dataset(full_data, train_ratio, validation_ratio, test_ratio):
    """
    Function that splits the dataset into train, validation, and test
    """
    random_idx = np.random.permutation(len(full_data))
    train_threshold = int(round(train_ratio*len(full_data)))
    validation_threshold = int(round((train_ratio+validation_ratio)*len(full_data)))
    
    train_set = full_data.iloc[random_idx[:train_threshold]]
    validation_set = full_data.iloc[random_idx[train_threshold:validation_threshold]]
    test_set = full_data.iloc[random_idx[validation_threshold:]]
    
    return train_set, validation_set, test_set
# train_set, validation_set, test_set = split_dataset(full_data, 0.8, 0.1, 0.1)
# train_set.to_csv("../data/train.csv", index=False)
# validation_set.to_csv("../data/validation.csv", index=False)
# test_set.to_csv("../data/test.csv", index=False)

# load dataset
def load_datasets(load_dir = "../data"):
    """
    Function that loads the dataset
    """
    train_set = pd.read_csv("../data/train.csv")
    validation_set = pd.read_csv("../data/validation.csv")
    test_set = pd.read_csv("../data/test.csv")
    return train_set, validation_set, test_set
train_set, validation_set, test_set = load_datasets()


In [39]:
i = 290
print train_set.iloc[i]["question1"]
print train_set.iloc[i]["question2"]

Did Hitler have children and where are they?
Did Hitler have any children?


np.unique(test_set["is_duplicate"])
np.sum(test_set["is_duplicate"])/float(len(test_set))